# SigOpt

Once again, the function is

$$f(x) = (x-3)*(x-2)*(x-1)*x*(x+2)*(x+3)$$

which has a global min at -2.6 and two other local minima.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
f = lambda x: (x-3)*(x-2)*(x-1)*x*(x+2)*(x+3)
plt.plot((np.array(range(620))-310)/100.,list(map(f, (np.array(range(620))-310)/100.)))
plt.plot((-3.1,3.1),(0,0))
plt.show()

Now we will use a method from [SigOpt](https://sigopt.com/), which is a private company that specializes in Bayesian optimization. The company claims to be 100x faster than open source solutions. Below we create a connection to sigopt and do the same initilization that we did for GPyOpt.

In [ ]:
from sigopt import Connection
conn = Connection(client_token="INSERT TOKEN")
import numpy as np

def foo(x):
    return (x-3)*(x-2)*(x-1)*x*(x+2)*(x+3)

x_init = np.array([[2],[0.5],[-2.9]]) 
y_init = foo(x_init)

Now, we create an experiment with our connection. We pass in the domain, but nothing else:

In [ ]:
experiment = conn.experiments().create(
    name='1D Function Example',
    parameters=[
        dict(name='x', type='double', bounds=dict(min=-3.3, max=3.3))
    ],
)

Below we get a suggestions from SigOpt, then we evaluate it:

In [ ]:
new_x = conn.experiments(experiment.id).suggestions().create()
new_y = -foo(new_x.assignments['x'])

Now we send our observation back to SigOpt, and they will prepare the next suggestion:

In [ ]:
conn.experiments(experiment.id).observations().create(
    suggestion = new_x.id,
    value = new_y,
)

Below, we run the optimization for 20 iterations. If we go to the [SigOpt](https://sigopt.com/) website, we can go to the page and refresh the dashboard as the expreiment runs.  

In [ ]:
import time

for _ in range(20):
    
    new_x = conn.experiments(experiment.id).suggestions().create()
    new_y = -foo(new_x.assignments['x'])
    
    conn.experiments(experiment.id).observations().create(
        suggestion = new_x.id,
        value = new_y,
    )
    
    time.sleep(3)

SigOpt has a few more customizable options than GPyOpt, and I will learn more about performance speed in the coming weeks. I'm hoping to build a wrapped that lets us alternate between to routines. Also, of note is that SigOpt maximizes, so you have to change your code slightly.